# Local Results Performance Heatmap
Using a city and state, generate surrounding zipcodes and run a SERP API query for each keyword and zipcode.<br>The results are then visualized on a heatmap to show performance of the client's website in the local results.<br><br>NOTE: The API runs queries against Google "Places" and is not measuring performance of local pack results.

In [1]:
VALUE_SERP_KEY = ''
VALUE_SERP_ENDPOINT='https://api.valueserp.com/search'

### Retrieve Surrounding Zipcodes

In [ ]:
city = "Santa Clarita"
state = "CA"
distance_threshold = 6  # in miles, adjust as needed

In [52]:
import pgeocode
import tqdm
import zipcodes
from geopy.distance import geodesic
from tqdm.notebook import tqdm

def get_surrounding_zipcodes(city, state, distance_threshold):

    city_zips = zipcodes.filter_by(city=city, state=state)

    if not city_zips:
        return []

    surrounding_zips = set()
    
    for zip_info in tqdm(city_zips, desc='Generating surrounding zipcodes'):
        origin = (zip_info['lat'], zip_info['long'])

        # all zip codes by state and city
        all_zips = zipcodes.list_all()

        for target_zip in all_zips:
            target = (target_zip['lat'], target_zip['long'])

            # calculate distance
            distance = geodesic(origin, target).miles

            if distance <= distance_threshold:
                surrounding_zips.add(target_zip['zip_code'])

    # keep original city zipcodes
    surrounding_zips.update([zip_info['zip_code'] for zip_info in city_zips])

    return list(surrounding_zips)

In [53]:
surrounding_zipcodes = get_surrounding_zipcodes(city, state, distance_threshold)
print(f"Surrounding zipcodes within {distance_threshold} miles of {city}, {state}:")
print(surrounding_zipcodes)

Generating surrounding zipcodes:   0%|          | 0/5 [00:00<?, ?it/s]

Surrounding zipcodes within 6 miles of Santa Clarita, CA:
['91354', '91387', '91322', '91383', '91380', '91382', '91386', '91390', '91381', '91351', '91321', '91350', '91355', '91310']


### Run SERP API Queries

In [13]:
import pandas as pd 

# enter keywords separated by new line 
kws= '''dentist
dentist near me
dentist santa clarita
santa clarita dentist
santa clarita dental'''

kws = kws.split('\n')
df = pd.DataFrame(columns=['keywords', 'zipcode'])

for kw in kws:
    for zipcode in surrounding_zipcodes:
        df = pd.concat([df, pd.DataFrame({'keywords': kw, 'zipcode': zipcode}, index=[0])], ignore_index=True)       
df.head()

,keywords,zipcode
0,dentist,91354
1,dentist,91387
2,dentist,91322
3,dentist,91383
4,dentist,91380


In [14]:
import requests

def run_serp_query(keyword, zipcode, state, country, search_type, client_title):
    params = {
        'api_key': VALUE_SERP_KEY,
        'search_type': search_type,
        'q': keyword,
        'location': f'{zipcode},{state},{country}',
        'gl': 'us',
        'hl': 'en',
        'num': '20',
    }
    
    response = requests.get(VALUE_SERP_ENDPOINT, params=params)
    organic_results = response.json().get('places_results', [])
    
    result = {f'pos {i+1}': result['title'] for i, result in enumerate(organic_results, start=1)}
    
    client_position = next((r['position'] for r in organic_results if client_title in r['title']), 
                           f'{client_title} not found in top 20 results')
    
    result[f'{client_title}_pos'] = client_position
    
    return pd.DataFrame([result])

In [15]:
import pandas as pd
tqdm.pandas()

client_domain = 'dentalcsv.com'
client_title = 'Aesthetic Dental'  # client name, found in listing title
search_type = 'places'  # local results


results = df.progress_apply(
    lambda row: run_serp_query(row['keywords'], row['zipcode'], 'California', 'United States', search_type, client_title), 
    axis=1
)

df_results = pd.concat(results.tolist(), ignore_index=True)

  0%|          | 0/70 [00:00<?, ?it/s]

In [16]:
merged_df = pd.concat([df, df_results], axis=1)
merged_df.to_csv('export/kw_zipcode_df_aesthetic_dental.csv', index=False, encoding='utf-8')
print('########### Exported DataFrame to CSV ###########')
merged_df.head()

########### Exported DataFrame to CSV ###########


,keywords,zipcode,pos 2,pos 3,pos 4,pos 5,pos 6,pos 7,pos 8,pos 9,...,pos 14,pos 15,pos 16,pos 17,pos 18,pos 19,pos 20,pos 21,Aesthetic Dental_pos,pos 22
0,dentist,91354,Santa Clarita Advanced Dentistry,Valencia Advanced Dentistry at Copperhill Smil...,Smile City Dental,Copper Canyon Dentistry,Crest Dental,Canyon Dental Group,Smilebody Holistic Dental Wellbeing (formerly ...,Go Dental,...,Valencia cosmetic and implant dentistry,Valencia Dental Group at Copper Hill,C Squared Dentistry,Noa Dental,Baywood Dental Group,White Crown Dental,Aesthetic Dental & Specialty Center,Valencia Dental Office,19,NaN
1,dentist,91387,Lost Canyon Family Dental: Clayton R. Clark DD...,Golden Canyon Dentistry,Adventure Dental and Orthodontics - Santa Clarita,Dentistry For Kids and Adults,Santa Clarita Advanced Dentistry,Harmony Dental Santa Clarita,Canyon Country Dental Group,Canyon Country Family Dentistry,...,Aesthetic Dental & Specialty Center,"Family Tree Dentistry, Sarah J. Phillips, D.D.S.",Perfect Smiles Dental Studio,Villar Dentistry & Orthodontics,A New Smile CC Dental Group,California Dental,Wellness Family Dental Care,Navarro Dentistry,13,NaN
2,dentist,91322,Three Sixty Dentistry,Newhall Dental Arts,Aesthetic Dental & Specialty Center,Santa Clarita Advanced Dentistry,SANTA CLARITA DENTAL CARE,"Dr. David S. Kim, DDS",Santa Clarita Family Dentistry,Smile Design: Santa Clarita,...,Enhancedentist,Zak Dental - Valencia,Villar Dentistry & Orthodontics,Smilebody Holistic Dental Wellbeing (formerly ...,Rex Baumgartner DDS,Smile Republic Pediatric Dentistry (previously...,Lyons Dental Care,Newhall Oral & Maxillofacial Surgery Group,3,NaN
3,dentist,91383,Santa Clarita Advanced Dentistry,Smilebody Holistic Dental Wellbeing (formerly ...,Copper Canyon Dentistry,Hasley Canyon Dental Group,Aesthetic Dental & Specialty Center,Canyon Dental Group,Crest Dental,Villar Dentistry & Orthodontics,...,Valencia Dental Group and Orthodontics,Santa Clarita Valley Dental Care,Valencia cosmetic and implant dentistry,C Squared Dentistry,iHeartDDS - Santa Clarita Dentist,Dr. Marvin Tong DDS,White Crown Dental,Valencia Advanced Dentistry at Copperhill Smil...,5,NaN
4,dentist,91380,Santa Clarita Advanced Dentistry,Lost Canyon Family Dental: Clayton R. Clark DD...,Harmony Dental Santa Clarita,Adventure Dental and Orthodontics - Santa Clarita,Golden Canyon Dentistry,Dentistry For Kids and Adults,Villar Dentistry & Orthodontics,Bright Now! Dental & Orthodontics,...,Copper Canyon Dentistry,Canyon Country Family Dentistry,Kidz Dental Care,Canyon Country Dental Group,Valencia Dental Group and Orthodontics,Valencia Advanced Dentistry at Copperhill Smil...,Crest Dental,Canyon Dental Group,9,NaN


### Visualize Results - Plotly

In [24]:
nomi = pgeocode.Nominatim('us')

merged_df['zipcode'] = merged_df['zipcode'].astype(str)
merged_df['Latitude'] = nomi.query_postal_code(merged_df['zipcode'].tolist()).latitude
merged_df['Longitude'] = nomi.query_postal_code(merged_df['zipcode'].tolist()).longitude

merged_df_not_ranking = merged_df[merged_df['Aesthetic Dental_pos'].str.contains('Aesthetic', na=False)]
merged_df_ranking = merged_df[~merged_df['Aesthetic Dental_pos'].str.contains('Aesthetic', na=False)]

In [26]:
merged_df_ranking = merged_df_ranking.copy()
merged_df_ranking['Aesthetic Dental_pos'] = merged_df_ranking['Aesthetic Dental_pos'].astype(int)
group_ranking = merged_df_ranking.groupby(['Latitude', 'Longitude']).agg({'Aesthetic Dental_pos': 'mean',
                                                                          'keywords': 'count',
                                                                          'zipcode': 'first'}).reset_index()

In [27]:
group_not_ranking = merged_df_not_ranking.groupby(['Latitude', 'Longitude']).agg({'keywords': 'count',
                                                                                  'zipcode': 'first'}).reset_index()
group_not_ranking['ranking_pos'] = 1000 # used for folium mapping

In [28]:
# function for improved visualization
def expand_coordinates(df, lat_col='Latitude', lon_col='Longitude', factor=2): 
    
    # calculate the center
    center_lat = df[lat_col].mean()
    center_lon = df[lon_col].mean()

    # center the coordinates
    df['centered_lat'] = df[lat_col] - center_lat
    df['centered_lon'] = df[lon_col] - center_lon

    # expand the range
    df['expanded_lat'] = df['centered_lat'] * factor
    df['expanded_lon'] = df['centered_lon'] * factor

    # shift back to original scale
    df['new_lat'] = df['expanded_lat'] + center_lat
    df['new_lon'] = df['expanded_lon'] + center_lon

    return df

expanded_df = expand_coordinates(group_ranking, factor=2)

In [29]:
def ranking_bucket(pos):
  if pos <= 3:
    return "top 3"
  elif pos <= 10:
    return "4-10"
  elif pos < 20:
    return "page 2"
  else:
    return "page 3+"

expanded_df['position_bucket'] = expanded_df['Aesthetic Dental_pos'].apply(ranking_bucket)

In [36]:
import plotly_express as px
import plotly.graph_objects as go
import datetime

date_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

px_colors = px.colors.qualitative

color_map_position_bucket = {
    'top 3': px_colors.Plotly[2],
    '4-10': px_colors.Plotly[0],
    'page 2': px_colors.Plotly[4],
    'page 3+': px_colors.T10[9],
}

legend_order = ['top 3', 
                '4-10', 
                'page 2', 
                'page 3+']

In [54]:
# density map with ranking locations, adjust as needed
fig = px.scatter_mapbox(
    expanded_df,
    lat='new_lat',
    lon='new_lon',
    color='position_bucket',
    size='keywords',
    size_max=30,
    color_continuous_scale="speed_r",
    zoom=10, # adjust as needed
    center=dict(lat=expanded_df['new_lat'].mean(), lon=expanded_df['new_lon'].mean()),
    mapbox_style='open-street-map',
    hover_name='zipcode',
    color_discrete_map=color_map_position_bucket,
    category_orders={'position_bucket': legend_order},
    text='zipcode'
)

# non-ranking locations with red markers, adjust as needed
fig.add_trace(go.Scattermapbox(
    lat=group_not_ranking['Latitude'],
    lon=group_not_ranking['Longitude'],
    mode='markers',
    marker=go.scattermapbox.Marker(
        size=50,
        color=px_colors.Plotly[1],
        opacity=0.8
    ),
    hoverinfo='text',
    text="Zipcode=" + group_not_ranking['zipcode'] + ", Keywords=" + group_not_ranking['keywords'].astype(str),
    name="Low Rankings",
    showlegend=True
)
)

# layout updates, configure as needed
fig.update_layout(
    mapbox=dict(
        zoom=10,
        center=dict(
            lat=expanded_df['new_lat'].mean(),
            lon=expanded_df['new_lon'].mean(),
        ),
        style="open-street-map",
    ),
    title=f"Dentist-related 'Near Me' Ranking Zipcodes ({city}, {state})",
    title_x=0.5,
    width=1200,
    height=700,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        title="Ranking Bucket"
    )
)

fig.write_image(f"plots/plotly_heatmap_{date_time}.png")
fig.show()

### Visualize Results - Folium (optional)

In [56]:
import folium
from folium.plugins import HeatMap
import branca.colormap as cm


loc = 'Dentist-related "Near Me" Ranking Zipcodes ({}, {})'.format(city, state)
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

# Initialize the map
folium_fig = folium.Map(
    location=[expanded_df['new_lat'].mean(), expanded_df['new_lon'].mean()],
    zoom_start=11
)

folium_fig.get_root().html.add_child(folium.Element(title_html))

# prepare the data
map_values = expanded_df[['new_lat', 'new_lon', 'Aesthetic Dental_pos']]
map_missing_values = group_not_ranking[['Latitude', 'Longitude', 'ranking_pos']]

data = map_values.values.tolist()
data_missing = map_missing_values.values.tolist()

data = [x for x in data if str(x[0]) != 'nan']
data_missing = [x for x in data_missing if str(x[0]) != 'nan']

# gradient for rankings
gradient = {
    0.0: '#4B0082',  # indigo (lower rankings)
    0.2: '#0000FF',
    0.4: '#00BFFF',
    0.6: '#00FF00',
    0.8: '#FFD700',
    1.0: '#FFA500'   # orange (higher rankings)
}

# HeatMap for ranking data
hm_ranking = HeatMap(data,
                     min_opacity=0.55,
                     max_opacity=0.9,
                     radius=35,
                     gradient=gradient,
                     tooltip='Ranking',
                     ).add_to(folium_fig)

# gradient for missing rankings
gradient_not_ranking = {
    0.0: '#FFCCCB',
    0.25: '#FF9999',
    0.5: '#FF6666',
    0.75: '#FF3333',
    1.0: '#FF0000',
}

# HeatMap for missing ranking data
hm_not_ranking = HeatMap(data_missing,
                         min_opacity=0.55,
                         max_opacity=0.9,
                         radius=35,
                         gradient=gradient_not_ranking,
                         tooltip='Ranking',
                         ).add_to(folium_fig)


# color legend
colormap = cm.LinearColormap(
    colors=['#0000FF', 
            '#0000FF', 
            '#00BFFF', 
            '#00FF00', 
            '#FFD700', 
            '#FFA500'],
    vmin=min(expanded_df['Aesthetic Dental_pos']),
    vmax=max(expanded_df['Aesthetic Dental_pos']),
    caption='Ranking Position for Localized "Dental" Searches (Red = Not Ranking)',

)

colormap.add_to(folium_fig)

# optional, add color markers for greater precision
for idx, row in expanded_df.iterrows():
    folium.CircleMarker(
        location=[row['new_lat'], row['new_lon']],
        radius=8,  # size of the circle
        color=colormap(row['Aesthetic Dental_pos']),  # color based on ranking position
        fill=True,
        fill_opacity=1,
        tooltip=f"Zipcode: {row['zipcode']}, Ranking: {row['Aesthetic Dental_pos']}",
    ).add_to(folium_fig)
    

for idx, row in group_not_ranking.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=8,  
        color='#FF0000',  # red
        fill=True,
        fill_opacity=1,
        tooltip=f"Zipcode: {row['zipcode']}, Keywords: {row['keywords']}",
    ).add_to(folium_fig)


    
# save html file
folium_fig.save(f'plots/folium_map_{date_time}.html')

folium_fig

ModuleNotFoundError: No module named 'selenium'